In [1]:
import torch
import time
from thop import profile, clever_format

from Alignment import Alignment
from HDR import HDR

/home/asim/.conda/envs/asimenv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/asim/.conda/envs/asimenv/lib/python3.13/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [2]:
alignment_model = Alignment(n_channel =8, out_channel = 4, embed_dim = 60, depths=[4, 4, 4])
alignment_model = alignment_model.cuda()
hdr_model = HDR(n_channel =8, out_channel = 4, embed_dim = 60, depths=[4, 4, 4])
hdr_model = hdr_model.cuda()

/home/asim/.conda/envs/asimenv/lib/python3.13/site-packages/torch/functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4314.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [3]:
H, W = 256, 256
x1 = torch.rand(1, 8, H, W).cuda()
x2 = torch.rand(1, 8, H, W).cuda()
x3 = torch.rand(1, 8, H, W).cuda()
fused = torch.rand(1, 60, H, W).cuda()
f2 = torch.rand(1, 60, H, W).cuda()

In [4]:
alignment_model.eval()

flopsalignment, paramsalignment = profile(alignment_model, inputs=(x1, x2, x3))
flopsalignment, paramsalignment = clever_format([flopsalignment, paramsalignment], "%.3f")

print(f"FLOPs for alignment: {flopsalignment}")
print(f"Parameters for alignmnet: {paramsalignment}")

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_relu() for <class 'torch.nn.modules.activation.LeakyReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_upsample() for <class 'torch.nn.modules.upsampling.Upsample'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
[INFO] Register count_softmax() for <class 'torch.nn.modules.activation.Softmax'>.
FLOPs for alignment: 59.703G
Parameters for alignmnet: 1.269M


In [5]:

for _ in range(10):
    _ = alignment_model(x1, x2, x3)

# Measure
torch.cuda.synchronize()
start = time.time()
with torch.no_grad():
    for _ in range(100):
        _ = alignment_model(x1, x2, x3)
torch.cuda.synchronize()
end = time.time()

print(f"Avg Inference Time: {(end - start)*1000/100:.3f} ms")


Avg Inference Time: 27.502 ms


In [8]:
hdr_model.eval()

flopshdr, paramshdr = profile(hdr_model, inputs=(fused, f2))
flopshdr, paramshdr = clever_format([flopshdr, paramshdr], "%.3f")

print(f"FLOPs for hdr: {flopshdr}")
print(f"Parameters for hdr: {paramshdr}")

[INFO] Register count_normalization() for <class 'torch.nn.modules.normalization.LayerNorm'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register count_softmax() for <class 'torch.nn.modules.activation.Softmax'>.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_relu() for <class 'torch.nn.modules.activation.LeakyReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
FLOPs for hdr: 44.974G
Parameters for hdr: 686.344K


In [7]:
for _ in range(10):
    _ = hdr_model(fused, f2)


# Measure
torch.cuda.synchronize()
start = time.time()
with torch.no_grad():
    for _ in range(100):
        _ = hdr_model(fused, f2)
torch.cuda.synchronize()
end = time.time()

print(f"Avg Inference Time: {(end - start)*1000/100:.3f} ms")

Avg Inference Time: 250.494 ms
